In [1]:
# As usual, a bit of setup
import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.cnn import *
from cs231n.data_utils import get_CIFAR10_data

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
data = get_CIFAR10_data()
for k, v in data.items():
  print('%s: ' % k, v.shape)

X_train:  (49000, 3, 32, 32)
y_train:  (49000,)
X_val:  (1000, 3, 32, 32)
y_val:  (1000,)
X_test:  (1000, 3, 32, 32)
y_test:  (1000,)


In [3]:
X_train = np.moveaxis(data['X_train'], 1, 3)
X_val = np.moveaxis(data['X_val'], 1, 3)
X_test = np.moveaxis(data['X_test'], 1, 3)
print(X_train.shape, X_val.shape, X_test.shape)

(49000, 32, 32, 3) (1000, 32, 32, 3) (1000, 32, 32, 3)


In [15]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import tensorflow as tf
import numpy as np


In [13]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [6]:
(X_train, y_train), (X_test, y_test) = (X_train, data['y_train']), (X_test, data['y_test']) # fetch CIFAR-10 data

In [7]:
num_train, height, width, depth = X_train.shape # there are 50000 training examples in CIFAR-10 
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

In [8]:
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_test) # Normalise data to [0, 1] range

In [9]:
Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [10]:
inp = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano)
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
# conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
# conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
# pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
# drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_1)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

In [16]:
model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
optimaizer =  tf.train.MomentumOptimizer(0.01, 0.9)
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer=optimaizer, # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

In [17]:
model.fit(X_train, Y_train,                # Train the model using the training set...
          batch_size=batch_size, epochs=1,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

Train on 44100 samples, validate on 4900 samples
Epoch 1/1
44100/44100 [==============================] - 183s 4ms/step - loss: 1.4374 - acc: 0.4863 - val_loss: 1.2358 - val_acc: 0.5890


In [18]:
model.evaluate(X_test, Y_test, verbose=1)  # Evaluate the trained model on the test set!

1000/1000 [==============================] - 1s 1ms/step


[1.2141429195404052, 0.59599999999999997]